# **01. Import Libraries and Load Data**

In [ ]:
pip install wrds --no-deps

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import wrds
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import kurtosis, skew

## Connect to WRDS

In [ ]:
# Establish a connection to the WRDS
db = wrds.Connection()

# **02. Data Collection**

## Select 50 Top Stocks

In [ ]:
# Get the earliest trading date for each permno
query_earliest_date = """
SELECT
    permno,
    MIN(date) as first_trade_date
FROM
    crsp.dsf
GROUP BY
    permno
HAVING
    MIN(date) <= '2000-01-01'
"""

earliest_dates = db.raw_sql(query_earliest_date)

# Ensure stocks are still active until December 31, 2024 (latest available date)
query_active_stocks = """
SELECT
    permno
FROM
    crsp.dsf
WHERE
    date BETWEEN '2000-01-01' AND '2024-12-31'
GROUP BY
    permno
HAVING
    COUNT(DISTINCT date) = (SELECT COUNT(DISTINCT date)
                            FROM crsp.dsf
                            WHERE date BETWEEN '2000-01-01' AND '2024-12-31')
"""

active_stocks = db.raw_sql(query_active_stocks)

# Combine the two sets of stocks to get those listed before 2000 and still active in 2024
filtered_permnos = earliest_dates.merge(active_stocks, on='permno', how='inner')

# Get the list of permnos as a comma-separated string
permnos_str = ','.join([str(permno) for permno in filtered_permnos['permno'].tolist()])

# Get market capitalisation, company name, and sector information for IT sector
query_main = f"""
SELECT
    a.permco,
    a.permno,
    a.date,
    a.shrout,
    a.prc * a.shrout as market_cap,
    b.shrcd,
    b.exchcd,
    b.siccd,
    b.ncusip,
    b.comnam,
    b.ticker
FROM
    crsp.dsf AS a
JOIN
    crsp.dsenames AS b
ON
    a.permno = b.permno
WHERE
    (
        (b.siccd BETWEEN 3570 AND 3579) OR  -- IT-related services (programming, software, etc.)
        (b.siccd BETWEEN 3600 AND 3674) OR
        (b.siccd BETWEEN 7370 AND 7379) OR
        (b.siccd BETWEEN 4810 AND 4813)
    )
    AND a.permno IN ({permnos_str})
    AND a.date = '2024-12-31'
    AND b.exchcd IN (1, 3)
"""

# Execute query
crsp_data = db.raw_sql(query_main)

In [ ]:
# Check the results from crsp_data
crsp_data.head()

In [ ]:
print("Original dataset size: ", len(crsp_data))
print("Original number of stocks: ", len(set(crsp_data['permno'])))

## Calculate summary statistics for market cap

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import kurtosis, skew

# Group by ticker and calculate basic statistics for market_cap
ticker_stats = crsp_data.groupby('ticker')['market_cap'].describe(percentiles=[.25, .5, .75])

# Define functions for calculating skewness and kurtosis safely
def safe_kurtosis(x):
    # Remove constant values
    if len(np.unique(x)) < 2:
        return np.nan  # No variation, cannot compute kurtosis
    return kurtosis(x, nan_policy='omit')

def safe_skew(x):
    # Remove constant values
    if len(np.unique(x)) < 2:
        return np.nan  # No variation, cannot compute skewness
    return skew(x, nan_policy='omit')

# Calculate kurtosis and skewness for each ticker, handle errors with safe functions
ticker_stats['kurtosis'] = crsp_data.groupby('ticker')['market_cap'].apply(safe_kurtosis)
ticker_stats['skewness'] = crsp_data.groupby('ticker')['market_cap'].apply(safe_skew)

# Calculate variance for each ticker
ticker_stats['variance'] = crsp_data.groupby('ticker')['market_cap'].var()

# Select and display only the desired statistics (Min, Max, Mean, STD, Kurtosis, Skewness, and Variance)
desired_stats = ticker_stats[['min', 'max', 'mean', 'std', 'variance', 'kurtosis', 'skewness']]

# Rename columns for clarity
desired_stats = desired_stats.rename(columns={
    'min': 'Min',
    'max': 'Max',
    'mean': 'Mean',
    'std': 'STD',
    'variance': 'Variance',
    'kurtosis': 'Kurtosis',
    'skewness': 'Skewness'
})

# Get the top 50 tickers by market capitalization (mean)
top_50_tickers = desired_stats.sort_values(by='Mean', ascending=False).head(50)

# Create plots for each of the statistics

import matplotlib.pyplot as plt
import seaborn as sns

# Set up the figure size for multiple plots
plt.figure(figsize=(25, 25))

# Plot for Min values
plt.subplot(4, 2, 1)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Min'], color='skyblue')
plt.xticks(rotation=90)
plt.title('Min (Market Capitalisation)')

# Plot for Max values
plt.subplot(4, 2, 2)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Max'], color='lightgreen')
plt.xticks(rotation=90)
plt.title('Max (Market Capitalisation)')

# Plot for Mean
plt.subplot(4, 2, 3)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Mean'], color='cyan')
plt.xticks(rotation=90)
plt.title('Mean (Market Capitalisation)')

# Plot for STD (Standard Deviation)
plt.subplot(4, 2, 4)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['STD'], color='pink')
plt.xticks(rotation=90)
plt.title('STD (Market Capitalisation)')

# Plot for Variance
plt.subplot(4, 2, 5)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Variance'], color='lightcoral')
plt.xticks(rotation=90)
plt.title('Variance (Market Capitalisation)')

# Plot for Skewness
plt.subplot(4, 2, 6)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Skewness'], color='lightblue')
plt.xticks(rotation=90)
plt.title('Skewness (Market Capitalisation)')

# Plot for Kurtosis
plt.subplot(4, 2, 7)
sns.barplot(x=top_50_tickers.index, y=top_50_tickers['Kurtosis'], color='yellow')
plt.xticks(rotation=90)
plt.title('Kurtosis (Market Capitalisation)')

# Adjust layout to prevent overlap
plt.tight_layout(pad=5.0)

# Show the plots
plt.show()

# Print the top 50 stats table
print("Top 50 Tickers Market Capitalisation Stats:")
display(top_50_tickers)

In [ ]:
# Filter data for the latest date
latest_date = crsp_data['date'].max()
latest_data = crsp_data[crsp_data['date'] == latest_date]

# Group by permco and permno and select the entry with the highest market capitalisation within each group
top_50_IT_stocks = latest_data.groupby(['permco', 'permno']).apply(lambda x: x.nlargest(1, 'market_cap'))

# Sort by market capitalization and get the top 50 stocks
top_50_IT_stocks = top_50_IT_stocks.sort_values(by='market_cap', ascending=False).head(50)
top_50_IT_stocks.reset_index(drop=True, inplace=True)

In [ ]:
print(top_50_IT_stocks)

## Word Cloud for Top 50 stocks (Tickers)

In [ ]:
pip install wordcloud matplotlib

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Get the tickers from the top 50 IT stocks
tickers = top_50_IT_stocks['ticker'].dropna().tolist()  # Drop any NaN tickers

# Join the tickers into a single string
tickers_string = ' '.join(tickers)

# Create the WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(tickers_string)

# Display the WordCloud
plt.figure(figsize=(10, 6))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Check for missing values in important columns
missing_data = crsp_data[crsp_data[['market_cap', 'comnam', 'ncusip', 'ticker']].isna().any(axis=1)]

# Display the rows with missing data
print(missing_data)

In [ ]:
# Before removing duplicates
print(f"Data size before removing duplicates: {crsp_data.shape}")

# Remove duplicates
crsp_data.drop_duplicates(subset=['permno', 'date', 'date'], keep='first', inplace=True)

# After removing duplicates
print(f"Data size after removing duplicates: {crsp_data.shape}")

## Collect Price and Return Data

In [ ]:
# Get permno of the top 50 stocks
top_50_permnos = top_50_IT_stocks['permno'].tolist()

# Convert permno list to a string for the SQL IN clause
permnos_str = ', '.join(map(str, top_50_permnos))

### Download train data



In [ ]:
# Define the date range
start_date = '2000-01-01'
end_date = '2015-12-31'

# Query to get data for the specified date range and variables for the top 50 stocks
query = f"""
SELECT
    a.permco,
    a.permno,
    b.comnam,
    b.ticker,
    a.date,
    a.prc,
    a.cfacpr,
    a.ret
FROM
    crsp.dsf AS a
JOIN
    (SELECT permno, comnam, ticker, namedt, nameendt
     FROM crsp.dsenames
     WHERE permno IN ({permnos_str}) -- filter for the top 50 stocks
       AND namedt <= '{end_date}'
       AND (nameendt IS NULL OR nameendt >= '{start_date}')) AS b
ON
    a.permno = b.permno
WHERE
    a.permno IN ({permnos_str})     -- filter for the top 50 stocks
    AND a.date BETWEEN '{start_date}' AND '{end_date}'
    AND a.date >= b.namedt
    AND (a.date <= b.nameendt OR b.nameendt IS NULL)
"""

# Execute query
crsp_train = db.raw_sql(query)
crsp_train.sort_values(by=['permco', 'date'], inplace=True)

In [ ]:
# Check for missing values
print(crsp_train.isna().sum())

In [ ]:
# Drop rows where 'prc' or 'ret' are missing (NaN)
crsp_train = crsp_train.dropna(subset=['prc', 'ret'])

In [ ]:
crsp_train

## Merge the risk-free rate with stock returns (calculate excess returns)

In [ ]:
# Query to fetch the daily risk-free rate for the period 2000-2015
query_risk_free = """
SELECT
    date,
    rf
FROM
    ff.factors_daily
WHERE
    date BETWEEN '2000-01-01' AND '2015-12-31'
"""
rf_data = db.raw_sql(query_risk_free)

# Ensure both 'date' columns are in datetime format before merging
crsp_train['date'] = pd.to_datetime(crsp_train['date'], errors='coerce')
rf_data['date'] = pd.to_datetime(rf_data['date'], errors='coerce')

# Merge the risk-free rate with stock data
crsp_train = pd.merge(crsp_train, rf_data, how='left', on='date')

# Adjust the returns by factoring in the price adjustment factor (cfacpr)
crsp_train['adjusted_ret'] = crsp_train['ret'] / crsp_train['cfacpr']

# Calculate excess returns using the adjusted returns
crsp_train['excess_ret'] = crsp_train['adjusted_ret'] - crsp_train['rf']

# Clip abnormal returns to +100% and -100%
crsp_train['excess_ret'] = crsp_train['excess_ret'].clip(lower=-1.0, upper=1.0)

# Convert the excess return to a binary target for directional forecasting
crsp_train['directional_target'] = np.where(crsp_train['excess_ret'] > 0, 1, 0)

# Check the results for train data
crsp_train[['permco', 'permno', 'date', 'adjusted_ret', 'excess_ret']].head()

## Download Test Data (2016-2024)

In [ ]:
# Define the date range
start_date = '2016-01-01'
end_date = '2024-12-31'

# Query to get data for the specified date range and variables for the top 50 stocks
query = f"""
SELECT
    a.permco,
    a.permno,
    b.comnam,
    b.ticker,
    a.date,
    a.prc,
    a.cfacpr,
    a.ret
FROM
    crsp.dsf AS a
JOIN
    (SELECT permno, comnam, ticker, namedt, nameendt
     FROM crsp.dsenames
     WHERE permno IN ({permnos_str}) -- filter for the top 50 stocks
       AND namedt <= '{end_date}'
       AND (nameendt IS NULL OR nameendt >= '{start_date}')) AS b
ON
    a.permno = b.permno
WHERE
    a.permno IN ({permnos_str})       -- filter for the top 50 stocks
    AND a.date BETWEEN '{start_date}' AND '{end_date}'
    AND a.date >= b.namedt
    AND (a.date <= b.nameendt OR b.nameendt IS NULL)
"""
# Execute query
crsp_test = db.raw_sql(query)
crsp_test.sort_values(by=['permco', 'date'], inplace=True)

In [ ]:
crsp_test

In [ ]:
# Check for missing values
print(crsp_test.isna().sum())

## Calculate Excess Returns for Test Data

In [ ]:
# Use the Fama French data to get the daily risk-free rate for the test period (2016-2024)
query_risk_free_test = """
SELECT
    date,
    rf
FROM
    ff.factors_daily
WHERE
    date BETWEEN '2016-01-01' AND '2024-12-31'
"""
rf_data_test = db.raw_sql(query_risk_free_test)

# Merge risk-free rate with test data
crsp_test['date'] = pd.to_datetime(crsp_test['date'], errors='coerce')
rf_data_test['date'] = pd.to_datetime(rf_data_test['date'], errors='coerce')

# Merge the test data with the risk-free rate data
crsp_test = pd.merge(crsp_test, rf_data_test, how='left', on='date')

# Adjust the returns by factoring in the price adjustment factor (cfacpr)
crsp_test['adjusted_ret'] = crsp_test['ret'] / crsp_test['cfacpr']

# Calculate excess returns using the adjusted returns
crsp_test['excess_ret'] = crsp_test['adjusted_ret'] - crsp_test['rf']

# Clip abnormal returns to +100% and -100%
crsp_test['excess_ret'] = crsp_test['excess_ret'].clip(lower=-1.0, upper=1.0)

# Convert the excess return to a binary target for directional forecasting
crsp_test['directional_target'] = np.where(crsp_test['excess_ret'] > 0, 1, 0)

# Check the results for test data
crsp_test[['permco', 'permno', 'date', 'adjusted_ret', 'excess_ret']].head()

### Adjusted ret and excess ret are similar because the risk-free rate (rf) is very close to zero around those years.

## Descriptive Statistics for Excess Returns

In [ ]:
# Calculate descriptive statistics for excess returns in the training dataset
in_sample_stats = crsp_train["excess_ret"].describe()

# Calculate skewness and kurtosis
skewness = stats.skew(crsp_train["excess_ret"])
kurtosis = stats.kurtosis(crsp_train["excess_ret"])

# Print the statistics in the desired format
print("In-Sample Excess Return Stats:")
print(in_sample_stats)

# Print skewness and kurtosis
print(f"Skewness: {skewness:.4f}")
print(f"Kurtosis: {kurtosis:.4f}")

# Display the dtype
print(f"Name: excess_ret, dtype: {crsp_train['excess_ret'].dtype}")

In [ ]:
# Calculate descriptive statistics for excess returns in the testing dataset
out_sample_stats = crsp_test["excess_ret"].describe()

# Calculate skewness and kurtosis
skewness = stats.skew(crsp_test["excess_ret"])
kurtosis = stats.kurtosis(crsp_test["excess_ret"])

# Print the statistics in the desired format
print("Out-Sample Excess Return Stats:")
print(out_sample_stats)

# Print skewness and kurtosis
print(f"Skewness: {skewness:.4f}")
print(f"Kurtosis: {kurtosis:.4f}")

# Display the dtype
print(f"Name: excess_ret, dtype: {crsp_test['excess_ret'].dtype}")

In [ ]:
train_stats = crsp_train.groupby('permno')['excess_ret'].describe()
test_stats = crsp_test.groupby('permno')['excess_ret'].describe()

# Print descriptive statistics
print("Descriptive Statistics for Excess Returns (Training Period):")
print(train_stats)

print("\nDescriptive Statistics for Excess Returns (Test Period):")
print(test_stats)

## Ceate Rolling Windows

In [ ]:
def create_lag_features(df, lags):
    # Sort the data by stock ID ('permno') and date to ensure correct time order
    df_sorted = df.sort_values(by=["permno", "date"])

    # Loop through each lag value provided (e.g., 5, 21, 252, 512)
    for lag in lags:
        # Create lag features by shifting excess returns and applying a rolling window
        df[f"lag_{lag}"] = (
            df_sorted.groupby("permno")["excess_ret"]  # Group by stock
            .shift(1)  # Shift by 1 day to avoid lookahead bias
            .rolling(window=lag, min_periods=1)  # Rolling window over past 'lag' days
            .mean()  # Calculate the mean of the rolling window
        )

    # Return the DataFrame with added lag features
    return df

# Example usage for both crsp_train and crsp_test
lag_days_list = [5, 21, 252, 512]  # Example list of lag days

# Apply the function to both crsp_train and crsp_test
crsp_train_lagged = create_lag_features(crsp_train, lag_days_list)
crsp_test_lagged = create_lag_features(crsp_test, lag_days_list)

# Drop rows where any of the lag columns are NaN in crsp_test_lagged
crsp_test_lagged = crsp_test_lagged.dropna(subset=[f'lag_{lag}' for lag in lag_days_list])

# Verify that the lag features are correctly added
print(crsp_train_lagged.head())
print(crsp_test_lagged.head())